# Dynamic Mode Decomposition
*(Based on the derivation in [Data-Driven Science and Engineering by Brunton and Kutz](http://www.databookuw.com))*


Dynamic mode decomposition (DMD) is a method of finding a low-rank representation of the Koopman matrix $A$ that operates on a sequence of measurement snapshot pairs $\{ \mathbf{x}(t_k), \mathbf{x}(t_k')\}$ from a dynamical system.
That is,
\begin{align}
\mathbf{x}(t_k') = A \mathbf{x}(t_k)
\end{align}

The snapshot pairs are assembled into matrices
\begin{align}
X &= [\mathbf{x}(t_1), \mathbf{x}(t_2), \dots, \mathbf{x}(t_m) ]\\
X' &= [\mathbf{x}(t_1'), \mathbf{x}(t_2'), \dots, \mathbf{x}(t_m') ]
\end{align}
so that
\begin{align}
X' = A X.
\end{align}
The full-rank matrix $A$ can be found directly using the pseudo-inverse,
\begin{align}
A = X' X^\dagger.
\end{align}

However, if the state dimension $n$ is very large, this could be computationally intractable, so we desire to find an approximation $\tilde{A}$ with rank $r \ll n$ that still captures the dominating dynamics of the system.
The DMD finds this approximation, and importantly, it finds the dominating dynamic modes (eigenvectors) $\Phi$ of the high dimensional system.

## DMD algorithm for $\tilde{A}$

1. Compute the rank-reduced SVD of $X$.

\begin{align}
X \approx \tilde{U} \tilde{\Sigma} \tilde{V}^H
\end{align}

2. Compute the reduced matrix $\tilde{A}$

\begin{align}
\tilde{A} = \tilde{U}^H X' \tilde{V} \tilde{\Sigma}^{-1}
\end{align}

## DMD algorithm for $\Phi$

3. Find the eigenvectors ($W$) and eigenvalues ($\Lambda$) of $\tilde{A}$.

\begin{align}
\tilde{A} W = W \Lambda
\end{align}

4. Compute the high-dimensional DMD modes

\begin{align}
\Phi = X' \tilde{V} \tilde{\Sigma}^{-1} W
\end{align}


In [52]:
using LinearAlgebra

function DMD_A(X, Xprime, r)
    U, S, V = svd(X, full=false)
    
    Utilde = U[:, 1:r]
    Stilde = S[1:r]
    Vtilde = V[:, 1:r]
    
    Atilde = Utilde' * Xprime * Vtilde * diagm(Stilde.^(-1))
    
    W = eigvecs(Atilde)
    Lambda = eigvals(Atilde)
    Phi = Xprime * Vtilde * diagm(Stilde.^-1) * W
    
    return Atilde, Phi, Lambda
end


DMD_A (generic function with 1 method)

In [48]:
A = [0. 1. 0.; 
     0. 0. -2.;
     .5 0. 0.]

x0 = [1., 0., 0.]

n = length(x0)
m = 10

X = zeros(n, m)

X[:,1] = x0

for i = 2:m
    X[:,i] = A * X[:,i-1]
end

X

3×10 Matrix{Float64}:
 1.0  0.0   0.0  -1.0   0.0  0.0  1.0  0.0   0.0  -1.0
 0.0  0.0  -1.0   0.0   0.0  1.0  0.0  0.0  -1.0   0.0
 0.0  0.5   0.0   0.0  -0.5  0.0  0.0  0.5   0.0   0.0

In [53]:
Atilde, Phi, Lambda = DMD_A(X[:,1:m-1], X[:,2:m], 3)
@show Atilde
@show Phi

Atilde = [0.0 0.0 -2.0; 1.0000000000000002 0.0 3.2049378106392735e-16; 0.0 0.5 0.0]
Phi = ComplexF64[-0.666666666666667 + 0.0im -0.6666666666666666 - 9.251858538542979e-17im -0.6666666666666666 + 9.251858538542979e-17im; 0.6666666666666669 + 0.0im -0.3333333333333327 + 0.5773502691896262im -0.3333333333333327 - 0.5773502691896262im; 0.3333333333333333 + 0.0im -0.16666666666666669 - 0.2886751345948128im -0.16666666666666669 + 0.2886751345948128im]


3×3 Matrix{ComplexF64}:
 -0.666667+0.0im  -0.666667-9.25186e-17im  -0.666667+9.25186e-17im
  0.666667+0.0im  -0.333333+0.57735im      -0.333333-0.57735im
  0.333333+0.0im  -0.166667-0.288675im     -0.166667+0.288675im

In [56]:
eigvecs(A)

3×3 Matrix{ComplexF64}:
 -0.666667+0.0im  0.666667-0.0im       0.666667+0.0im
  0.666667+0.0im  0.333333-0.57735im   0.333333+0.57735im
  0.333333+0.0im  0.166667+0.288675im  0.166667-0.288675im

In [57]:
Phi

3×3 Matrix{ComplexF64}:
 -0.666667+0.0im  -0.666667-9.25186e-17im  -0.666667+9.25186e-17im
  0.666667+0.0im  -0.333333+0.57735im      -0.333333-0.57735im
  0.333333+0.0im  -0.166667-0.288675im     -0.166667+0.288675im

In [66]:
@show A * x0
@show (Phi) * diagm(Lambda) * pinv(Phi) * x0

A * x0 = [0.0, 0.0, 0.5]
Phi * diagm(Lambda) * pinv(Phi) * x0 = ComplexF64[8.881784197001252e-16 + 0.0im, 1.6653345369377348e-16 + 1.1102230246251565e-16im, 0.5000000000000004 - 4.1368802094960244e-17im]


3-element Vector{ComplexF64}:
  8.881784197001252e-16 + 0.0im
 1.6653345369377348e-16 + 1.1102230246251565e-16im
     0.5000000000000004 - 4.1368802094960244e-17im